In [1]:
from pathlib import Path
import re
import pandas as pd
from IPython.display import Audio, display
from pydub import AudioSegment
import numpy as np
from textwrap import wrap
from time import sleep

In [2]:
pattern = re.compile(r"^[sS]tortinget-(\d{4}).*")

def prepare_segment(row, audiofolder):
    audio = list(row.audiofilename)[0] + ".mp3"
    year = pattern.match(audio).group(1)
    segment = AudioSegment.from_mp3(Path(audiofolder) / year / audio)
    segment.export("test_clip.wav", format='wav')
    print(f"File: {audio}\n")
    print(f"Language: {list(row.language)[0]}")
    print("corpus:\n" + '\n'.join(wrap(list(row.proceedings_text)[0], width=100)) + "\n")
    print(f"asr:\n" + '\n'.join(wrap(list(row.transcription_text)[0], width=100)) + "\n")


In [3]:
audiofolder = "/media/pers/elements/npscpluss_audio/segmented"

In [4]:
annotation_df = pd.read_csv("data/annotated_dataset.csv")

In [5]:
def get_annotation(row):
    row.missing_start_word = False
    row.missing_end_word = False
    row.wrong_match = False
    row.wrong_language = False
    row.low_quality = False
    if input("Missing start word?") != "":
        row.missing_start_word = True
    if input("Missing end word") != "":
        row.missing_end_word = True
    if input("Wrong match") != "":
        row.wrong_match = True
    if input("Wrong language") != "":
        row.wrong_language = True
    if input("Low quality") != "":
        row.low_quality = True
    return row

In [6]:
def annotate_data(filename, audiofolder):
    df = pd.read_csv(filename)
    updated_cells = [
        "missing_start_word",
        "missing_end_word",
        "wrong_match",
        "wrong_language",
        "low_quality",
    ]
    subset_df = df[df.missing_start_word.isna()]
    random_row = subset_df.sample()
    prepare_segment(random_row, audiofolder)
    display(Audio("test_clip.wav", autoplay=True))
    random_row = get_annotation(random_row)
    df.loc[random_row.index] = random_row
    df.to_csv(filename, index=False)


    

In [7]:
annotate_data("data/annotated_dataset.csv", audiofolder)

File: Stortinget-20220118-095509_5586400_5600300.mp3

score: 0.7916666666666666
Language: bm
corpus:
Det er sakens realiteter. Vi ønsker framdrift og realisering av nye sykehusbygg til beste for
pasientene i Oslo. Etter å ha hørt statsråden er

asr:
så det er sakens realiteter vi ønsker framdrift og realisering av nye sykehusbygg til beste for
pasientene i oslo neste taler er bård hoksrud

